# 📢📢📢 Projet NLP : La parole aux citoyens (Make.org - Le tourisme vert en Ille-et-Vilaine)📢📢📢
# Lecture et nettoyage des données
## Importation des bibliothèques nécessaires

In [1]:
import pandas as pd
import re
import emoji
import os

## Lecture des données

In [2]:
data = pd.read_json("../data/raw/ensai_2024_req_tourisme_responsable.json")
data.head()

,total,results,seed
0,1496,"{'id': '67d0c562-fb00-4228-aa08-e504a8077c38',...",NaN
1,1496,"{'id': '9798263d-96a3-4e35-9ba2-f15c6f6402d8',...",NaN
2,1496,"{'id': '885d953e-a29a-4323-96d8-f149d3e512c7',...",NaN
3,1496,"{'id': '6a660958-f5c7-44ce-83de-d165263c8986',...",NaN
4,1496,"{'id': 'fb259cf7-ec19-459b-a6ad-2254689b14c3',...",NaN


## Arborescence et organisation du fichier json

In [3]:
data["results"].iloc[0]

{'id': '67d0c562-fb00-4228-aa08-e504a8077c38',
 'userId': '93ef60de-5827-49f1-bd56-e1f580619540',
 'content': "Il faut favoriser le stationnement des véhicules de loisir proche des sites touristiques au même titre qu'un véhicule léger.",
 'contentLanguage': 'fr',
 'translatedContent': None,
 'translatedLanguage': None,
 'slug': 'il-faut-favoriser-le-stationnement-des-vehicules-de-loisir-proche-des-sites-touristiques-au-meme-titre-qu-un-vehicule-leger',
 'status': 'Accepted',
 'createdAt': '2021-06-04T08:36:40.430Z',
 'updatedAt': '2021-08-19T12:29:37.892Z',
 'votes': [{'voteKey': 'agree',
   'count': 46,
   'score': 0.37,
   'qualifications': [{'qualificationKey': 'likeIt',
     'count': 8,
     'hasQualified': False},
    {'qualificationKey': 'platitudeAgree', 'count': 2, 'hasQualified': False},
    {'qualificationKey': 'doable', 'count': 11, 'hasQualified': False}],
   'hasVoted': False},
  {'voteKey': 'neutral',
   'count': 38,
   'score': 0.37,
   'qualifications': [{'qualification

## Sélection et réorganisation en dataframe

On ne va sélectionner que les informations :
- Content : le contenu de la proposition citoyenne
- contentLanguage : la langue dans laquelle la proposition est écrite
- votes :
    - votekey : agree, disagree, neutral
        - count : le décompte du nombre de votes soit agree, soit disagree, soit neutral
        - qualificationkey : correspond à des sous-votes
            - likeIt, platitudeagree, doable, ce sont les sous-catégories de agree
            - noOpinion, doNotUnderstand, doNotCare, ce sont les sous-catégories de neutral
            - impossible, noWay, platitudeDisagree, ce sont les sous-catégories de disagree
            - count : le decompte pour chque sous vote, quand il y en a
NB : platitudeagree, c'est banalité

### Traitement pour les votes

In [4]:
votes = pd.json_normalize(
    data["results"],
    record_path=["votes"],
    meta=["id", "content", "contentLanguage"]
)

# Pivot pour obtenir une colonne par voteKey (agree.count, agree.score, ...)
votes = votes.pivot_table(
    index=["id", "content", "contentLanguage"],
    columns="voteKey",
    values=["count", "score"],
    aggfunc="first"
).reset_index()

# Renommer colonnes (count.agree → agree.count)
votes.columns = [
    f"{col[1]}_{col[0]}" if col[1] else col[0]
    for col in votes.columns
]


In [5]:
votes.head(3)

,id,content,contentLanguage,agree_count,disagree_count,neutral_count,agree_score,disagree_score,neutral_score
0,001b47e6-e6da-4850-859d-311e213924cf,Il faut une surveillance des lieux touristique...,fr,27,7,7,0.65,0.18,0.17
1,0087d988-b571-4e81-b68d-44d259dd61b4,Il faut donner des sacs poubelles et des cendr...,fr,56,18,15,0.62,0.19,0.19
2,00a3341d-4463-4660-a516-2f5a0237abe6,Il faut récompenser et mettre en avant les act...,fr,35,7,14,0.62,0.13,0.25


### Traitement pour les qualifications

In [6]:
qualif = pd.json_normalize(
    data['results'],
    record_path=["votes", "qualifications"],
    meta=['id']
)

# Pivot qualifications
qualif = qualif.pivot_table(
    index=["id"],
    columns=["qualificationKey"],
    values="count",
    aggfunc="first"
).reset_index()

In [7]:
qualif.head(3)

qualificationKey,id,doNotCare,doNotUnderstand,doable,impossible,likeIt,noOpinion,noWay,platitudeAgree,platitudeDisagree
0,001b47e6-e6da-4850-859d-311e213924cf,1,0,8,2,2,0,3,3,4
1,0087d988-b571-4e81-b68d-44d259dd61b4,0,0,21,6,6,5,4,5,7
2,00a3341d-4463-4660-a516-2f5a0237abe6,0,5,12,2,2,2,3,1,5


## Jointure (utilisation de merge) des votes et qualifications sur les ids

In [8]:
df = votes.merge(
    qualif,
    left_on=["id"],
    right_on=["id"]
)

In [9]:
df.head(3)

,id,content,contentLanguage,agree_count,disagree_count,neutral_count,agree_score,disagree_score,neutral_score,doNotCare,doNotUnderstand,doable,impossible,likeIt,noOpinion,noWay,platitudeAgree,platitudeDisagree
0,001b47e6-e6da-4850-859d-311e213924cf,Il faut une surveillance des lieux touristique...,fr,27,7,7,0.65,0.18,0.17,1,0,8,2,2,0,3,3,4
1,0087d988-b571-4e81-b68d-44d259dd61b4,Il faut donner des sacs poubelles et des cendr...,fr,56,18,15,0.62,0.19,0.19,0,0,21,6,6,5,4,5,7
2,00a3341d-4463-4660-a516-2f5a0237abe6,Il faut récompenser et mettre en avant les act...,fr,35,7,14,0.62,0.13,0.25,0,5,12,2,2,2,3,1,5


## Filtrage des propositions françaises et suppressions des colonnes inutiles (id et contentLanguage)

In [10]:
df['contentLanguage'].value_counts()

contentLanguage
fr    1493
en       3
Name: count, dtype: int64

In [11]:
df = df[df['contentLanguage'] == 'fr'].drop(columns=['id', 'contentLanguage'])

## Création de plusieurs colonnes :
- colonne pour le decompte de "?"
- colonne pour le decompte de "!"
- colonne pour le decompte de smiley

In [12]:
# Compter le nombre de '?' dans chaque élément de la colonne 'content'
df['nb_interrogations'] = df['content'].str.count(r'\?')

# Compter le nombre de '!' dans chaque élément de la colonne 'content'
df['nb_exclamation'] = df['content'].str.count(r'!')

### Petite fonction pour le cas de figure des emoji

In [13]:
def compter_emojis_smileys(texte:str) -> int:
    # Compter les emojis Unicode avec le package emoji
    nb_unicode = emoji.emoji_count(texte)
    
    # Compter les smileys ASCII classiques (ex: :), ;-), :D, etc.)
    ascii_smiley_regex = r'[:;=8][-^]?[)DdpP3/\\(|]'
    nb_ascii = len(re.findall(ascii_smiley_regex, texte))
    
    # Compter les emojis personnalisés type :nom:
    custom_emoji_regex = r':[^\s:]{1,20}:'
    nb_custom = len(re.findall(custom_emoji_regex, texte))
    
    # Total
    nb_total = nb_unicode + nb_ascii + nb_custom

    return nb_total

In [14]:
# Compter le nombre d'emoji
df['nb_emoji'] = df['content'].apply(compter_emojis_smileys)

In [15]:
df.head(3)

,content,agree_count,disagree_count,neutral_count,agree_score,disagree_score,neutral_score,doNotCare,doNotUnderstand,doable,impossible,likeIt,noOpinion,noWay,platitudeAgree,platitudeDisagree,nb_interrogations,nb_exclamation,nb_emoji
0,Il faut une surveillance des lieux touristique...,27,7,7,0.65,0.18,0.17,1,0,8,2,2,0,3,3,4,0,0,0
1,Il faut donner des sacs poubelles et des cendr...,56,18,15,0.62,0.19,0.19,0,0,21,6,6,5,4,5,7,0,0,0
2,Il faut récompenser et mettre en avant les act...,35,7,14,0.62,0.13,0.25,0,5,12,2,2,2,3,1,5,0,0,0


In [16]:
display(df['nb_interrogations'].value_counts())
display(df['nb_exclamation'].value_counts())
display(df['nb_emoji'].value_counts())

nb_interrogations
0    1483
1       9
2       1
Name: count, dtype: int64

nb_exclamation
0    1454
1      34
2       4
3       1
Name: count, dtype: int64

nb_emoji
0    1493
Name: count, dtype: int64

## Suppression du décompte de smiley (aucun smiley) et dedoublonnage préventif avant sauvegarde

In [17]:
df = df.drop(columns='nb_emoji', axis = 1).drop_duplicates()
df.shape

(1493, 18)

## Sauvegarde du dataframe

In [18]:
# Repertoire de sauvegarde
savepath = "../data/processed"

# Test de l'existence du répertoire
if not os.path.exists(savepath):
    # Création du répertoire s'il n'existe pas
    os.makedirs(savepath)

# Sauvegarde des jeux de données
df.to_csv(
    path_or_buf=os.path.join(savepath, "req_tourisme_responsable.csv"),
    sep=";",
    encoding="utf-8",
    index_label=False
)